In [81]:
"""module docstring"""
import pandas as pd
from itertools import permutations
import csv
from typing import List, Dict
from fractions import Fraction as ratio
from random import randint, sample

from entities import Bet, Competitor, Forecaster, Game
from boards import Boards
from utils import show


In [6]:
with open("players.csv") as tsv:
    vals = {}
    for column in zip(*[line for line in csv.reader(tsv, delimiter='\t')]):
        vals[column[0]] = list(column[1:])


trn = Boards(vals['player_name'], vals['username'], vals['program'])

def mark_w_random_wins(n: int):
    assert 3*n < trn.games_df.shape[0]
    
    for k in range(n): 
        trn.mark_winner(k, 'black')
        trn.mark_winner(k+n, 'draw')
        trn.mark_winner(k+2*n, 'white')
    pass

mark_w_random_wins(9)

trn.games_df.applymap(show).head()

,black,white,winner
1,Ray Lomeli,Daniel Harris,Ray Lomeli
2,Austin Cole,Mychal Hall,Austin Cole
3,Chris Schröder,Daniel Harris,Chris Schröder
4,Austin Cole,Chris Schröder,Austin Cole
5,Daniel Harris,Ryan Allred,Daniel Harris


In [7]:
trn.players_df

,score
Austin Cole,3.0
Chris Schröder,4.0
Crawford Collins,1.0
Daniel Harris,2.5
Huthman King,1.5
Mychal Hall,0.5
Pierre Damiba,1.5
Ray Lomeli,2.0
Ryan Allred,3.0
Scott Schindler,1.0


In [104]:
idspace=4

class Book(Boards): 
    def __init__(self, competitors: str, usernames: str, sections: str):
        super().__init__(competitors, usernames, sections)
        self.forecasters = {'HOUSE': Forecaster('HOUSE', balance=1000)}
        self.bets = self.init_H0()
        self.bettors = self.bettors_df(self.forecasters)
        self.book_df = self.make_bets_df()
        
    def make_forecaster(self, name: str, username='', program='', balance=100):
        """we can admit new forecasters on a rolling basis"""
        self.forecasters[name] = Forecaster(name, username, program, balance)
        self.bettors = self.bettors_df(self.forecasters)
        pass
    
    def bettors_df(self, bettors: Dict[str, Forecaster]):
        '''represent the table of bettors'''
        dat = {name: [bettors[name].name, bettors[name].username] for name in bettors.keys()}
        bettorsdf = pd.DataFrame.from_dict(dat, 
                                           orient='index', 
                                           columns=['balance', 'username'])
        return bettorsdf
    
    def init_H0(self) -> List[Bet]:
        ''' The house initializes each player at a null hypothesis. '''
        identifiers = [randint(10**idspace,10**(idspace+1)) for _ in range(len(self.competitors))]
        while len(set(identifiers))!=len(identifiers): 
            identifiers = [randint(10**idspace,10**(idspace+1)) for _ in range(len(self.competitors))]
        bets = {identi: Bet(ident=randint(10**idspace,10**(idspace+1)), 
                    posted_by=self.forecasters['HOUSE'], 
                    odds=ratio(1,1), on=name, amount=2) 
                for identi, name in zip(identifiers, self.competitors.values())}
        return bets
    
    def make_bets_df(self): 
        ''' shows a df of the bets'''
        dat = {b.ident: [b.posted_by, b.odds, b.on, b.amount, b.taken_by] for b in self.bets.values()}
        book_df = pd.DataFrame.from_dict(dat, 
                                         orient='index', 
                                         columns=['posted_by', 'odds', 'on', 'amount', 'taken_by'])
        return book_df
    
    def show_avail_bets(self): 
        '''returns a DF of bets, hiding the ones that have already been taken '''
        return self.book_df[self.book_df.taken_by.apply(lambda f: f.name)=="NOBODY"].applymap(show)
    
    def show_taken_bets(self):
        '''returns a DF of bets, ONLY the ones that have already been taken '''
        return self.book_df[self.book_df.taken_by.apply(lambda f: f.name)!="NOBODY"].applymap(show)
    
    def post_bet(self, posted_by, odds, on, amount): 
        x =randint(10**idspace,10**(idspace+1))
        while x in self.book_df.index: 
            x =randint(10**idspace,10**(idspace+1))
        assert x not in self.book_df.index
        self.bets[x] = (Bet(ident=x, 
                            posted_by=self.forecasters[posted_by], 
                            odds=odds, 
                            on=self.competitors[on], 
                            amount=amount))
        self.book_df = self.make_bets_df()
        pass
    
    def take_bet(self, by, ident): 
        taken_by = self.forecasters[by]
        assert isinstance(taken_by, Forecaster)
        self.bets[ident].taken_by = taken_by
        self.book_df = self.make_bets_df()
        
tourn = Book(vals['player_name'], vals['username'], vals['program'])

tourn.make_forecaster("angela", 'angela')

tourn.make_forecaster("hermione")

tourn.make_forecaster("dma", 'dma')

tourn.post_bet('dma', ratio(4,1), "Ryan Allred", 4)

tourn.post_bet('dma', ratio(9,1), "Troy Bradley", 1)

tourn.post_bet('hermione', ratio(3,1), "Pierre Damiba", 9)

tourn.book_df.applymap(show)

,posted_by,odds,on,amount,taken_by
59814,"(HOUSE, 1000)",1:1,Austin Cole,2,"(NOBODY, 0)"
81698,"(HOUSE, 1000)",1:1,Chris Schröder,2,"(NOBODY, 0)"
85841,"(HOUSE, 1000)",1:1,Crawford Collins,2,"(NOBODY, 0)"
31572,"(HOUSE, 1000)",1:1,Daniel Harris,2,"(NOBODY, 0)"
93370,"(HOUSE, 1000)",1:1,Huthman King,2,"(NOBODY, 0)"
74269,"(HOUSE, 1000)",1:1,Mychal Hall,2,"(NOBODY, 0)"
19115,"(HOUSE, 1000)",1:1,Pierre Damiba,2,"(NOBODY, 0)"
58564,"(HOUSE, 1000)",1:1,Ray Lomeli,2,"(NOBODY, 0)"
54671,"(HOUSE, 1000)",1:1,Ryan Allred,2,"(NOBODY, 0)"
87543,"(HOUSE, 1000)",1:1,Scott Schindler,2,"(NOBODY, 0)"


In [106]:
x = sample(list(tourn.bets.keys()), 9) 
# inexplicably, this is NOT equivalent to tourn.book_df.index. WTF. 

tourn.take_bet('hermione', x[0])

tourn.take_bet('dma', x[1])

tourn.take_bet('hermione', x[2])

tourn.take_bet('hermione', x[3])

tourn.take_bet('angela', x[4])

tourn.book_df.applymap(show)

,posted_by,odds,on,amount,taken_by
59814,"(HOUSE, 1000)",1:1,Austin Cole,2,"(angela, 100)"
81698,"(HOUSE, 1000)",1:1,Chris Schröder,2,"(hermione, 100)"
85841,"(HOUSE, 1000)",1:1,Crawford Collins,2,"(NOBODY, 0)"
31572,"(HOUSE, 1000)",1:1,Daniel Harris,2,"(hermione, 100)"
93370,"(HOUSE, 1000)",1:1,Huthman King,2,"(NOBODY, 0)"
74269,"(HOUSE, 1000)",1:1,Mychal Hall,2,"(hermione, 100)"
19115,"(HOUSE, 1000)",1:1,Pierre Damiba,2,"(NOBODY, 0)"
58564,"(HOUSE, 1000)",1:1,Ray Lomeli,2,"(hermione, 100)"
54671,"(HOUSE, 1000)",1:1,Ryan Allred,2,"(dma, 100)"
87543,"(HOUSE, 1000)",1:1,Scott Schindler,2,"(NOBODY, 0)"
